# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd

In [9]:
# import modin.pandas as pd
# from distributed import Client
# client = Client()

In [2]:
# leading data
df = pd.read_csv('./zillow_data.csv')

In [3]:
# display dataset
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


# Step 2: Data Preprocessing

In [4]:
# def get_datetimes(df):
#     return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [5]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [7]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [8]:
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


# Step 3: EDA and Visualization

In [12]:
import matplotlib.pyplot as plt
import matplotlib

In [13]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [14]:
df.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04'],
      dtype='object', length=272)

In [7]:
zipcode_list = df_melted['RegionID'].unique().tolist()
zipcode_list

[84654,
 90668,
 91982,
 84616,
 93144,
 91733,
 61807,
 84640,
 91940,
 97564,
 62037,
 71831,
 62087,
 62045,
 74101,
 96107,
 84646,
 74242,
 92593,
 84620,
 61703,
 92036,
 92045,
 69816,
 93123,
 92515,
 97771,
 71067,
 66126,
 96027,
 92271,
 92551,
 66133,
 90654,
 61802,
 74126,
 91922,
 86026,
 84615,
 91968,
 70829,
 89925,
 91685,
 91926,
 61803,
 62040,
 84630,
 62020,
 84627,
 70828,
 90637,
 62023,
 90823,
 69823,
 94748,
 60545,
 62025,
 66843,
 72463,
 92481,
 67732,
 91726,
 97711,
 70874,
 92617,
 99534,
 91889,
 90795,
 62032,
 92341,
 62039,
 61148,
 84621,
 96015,
 97565,
 92598,
 70810,
 84612,
 77007,
 90665,
 72363,
 72578,
 72442,
 62088,
 96025,
 79054,
 58669,
 67730,
 96239,
 87074,
 76953,
 98325,
 96193,
 62044,
 96990,
 95048,
 92654,
 62067,
 61790,
 71005,
 71864,
 74156,
 97764,
 62026,
 96007,
 94820,
 96545,
 95306,
 69672,
 60081,
 92336,
 95309,
 95804,
 94890,
 62022,
 65133,
 97431,
 93263,
 79040,
 94870,
 92520,
 97740,
 64180,
 98304,
 76511,


In [8]:
TS = {}
for zipcode in zipcode_list:
    temp_df = df_melted.groupby('RegionID').get_group(zipcode).sort_index()['value']
    TS[zipcode] = temp_df

To request implementation, send an email to feature_requests@modin.org.


KeyboardInterrupt: 

In [ ]:
TS.keys()

In [ ]:

ts_df = pd.DataFrame(TS)
ts_df.head()

# Step 4: Reshape from Wide to Long Format

In [8]:
# def melt_data(df):
#     melted = pd.melt(
#         df,
#         id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'],
#         var_name='time')
#     melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
#     melted = melted.dropna(subset=['value'])
#     return melted.groupby('time').aggregate({'value': 'mean'})

# melt_data(df.drop(labels=['RegionID', 'SizeRank'],axis=1))

In [50]:
# def melt_data(df):
#     melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
#     melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
#     melted = melted.dropna(subset=['value'])
#     return melted#melted.groupby('time').aggregate({'value':'mean'})

# melt_data(df.drop(labels=['RegionID', 'SizeRank'],axis=1))

In [4]:
def melt_data(df):
    melted = pd.melt(df,
                     id_vars=[
                         'RegionID', 'SizeRank', 'RegionName', 'City', 'State',
                         'Metro', 'CountyName'
                     ],
                     var_name='date')
    melted['date'] = pd.to_datetime(melted['date'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [5]:
df_melted = melt_data(df)
df_melted

Order of rows could be different from pandas.


,RegionID,SizeRank,RegionName,City,State,Metro,CountyName,date,value
0,84654,1,60657,Chicago,IL,Chicago,Cook,1996-04-01,334200.0
1,90668,2,75070,McKinney,TX,Dallas-Fort Worth,Collin,1996-04-01,235700.0
2,91982,3,77494,Katy,TX,Houston,Harris,1996-04-01,210400.0
3,84616,4,60614,Chicago,IL,Chicago,Cook,1996-04-01,498100.0
4,93144,5,79936,El Paso,TX,El Paso,El Paso,1996-04-01,77300.0
...,...,...,...,...,...,...,...,...,...
3901590,58333,14719,1338,Ashfield,MA,Greenfield Town,Franklin,2018-04-01,209300.0
3901591,59107,14720,3293,Woodstock,NH,Claremont,Grafton,2018-04-01,225800.0
3901592,75672,14721,40404,Berea,KY,Richmond,Madison,2018-04-01,133400.0
3901593,93733,14722,81225,Mount Crested Butte,CO,NaN,Gunnison,2018-04-01,664400.0


In [6]:
len(df_melted['RegionID'].unique())

14723

In [7]:
print(len(df_melted['State'].unique()))
df_melted['State'].unique()

51


array(['IL', 'TX', 'NY', 'CA', 'FL', 'TN', 'NC', 'GA', 'DC', 'MO', 'OK',
       'AZ', 'NJ', 'MD', 'VA', 'WA', 'OH', 'MI', 'MA', 'KS', 'NM', 'CT',
       'NV', 'PA', 'CO', 'OR', 'IN', 'SC', 'KY', 'AR', 'MN', 'AL', 'DE',
       'LA', 'MS', 'ID', 'MT', 'HI', 'WI', 'UT', 'ME', 'SD', 'WV', 'IA',
       'RI', 'NE', 'WY', 'AK', 'NH', 'VT', 'ND'], dtype=object)

In [8]:
print(len(df_melted['Metro'].unique()))
df_melted['Metro'].unique()

702


array(['Chicago', 'Dallas-Fort Worth', 'Houston', 'El Paso', 'New York',
       'San Francisco', 'The Villages', 'Nashville',
       'Los Angeles-Long Beach-Anaheim', 'Austin', 'Charlotte', 'McAllen',
       'Atlanta', 'Washington', 'San Antonio', 'Clarksville', 'St. Louis',
       'Oklahoma City', 'Phoenix', 'Baltimore', 'Miami-Fort Lauderdale',
       'Brownsville', 'Virginia Beach', 'Seattle', 'Cleveland',
       'Ann Arbor', 'Boston', 'Kansas City', 'Sacramento', 'Tucson',
       'Jacksonville', 'Napa', 'San Diego', 'Albuquerque', 'Hartford',
       'Las Vegas', 'Lancaster', 'Fresno', 'Denver', 'Detroit', 'Vallejo',
       'Pittsburgh', 'Columbus', 'Portland', 'Riverside', 'Yuma',
       'Ithaca', 'Springfield', 'Fort Myers', nan, 'Cincinnati', 'Tampa',
       'Columbia', 'Lafayette-West Lafayette', 'Fayetteville', 'Raleigh',
       'Kennewick', 'College Station', 'Hagerstown', 'Philadelphia',
       'Richmond', 'Indianapolis', 'Tulsa', 'Orlando', 'Ventura',
       'San Jose', 'Fin

In [9]:
df_melted.drop(columns=['RegionID', 'SizeRank'], inplace=True)
df_melted

,RegionName,City,State,Metro,CountyName,date,value
0,60657,Chicago,IL,Chicago,Cook,1996-04-01,334200.0
1,75070,McKinney,TX,Dallas-Fort Worth,Collin,1996-04-01,235700.0
2,77494,Katy,TX,Houston,Harris,1996-04-01,210400.0
3,60614,Chicago,IL,Chicago,Cook,1996-04-01,498100.0
4,79936,El Paso,TX,El Paso,El Paso,1996-04-01,77300.0
...,...,...,...,...,...,...,...
3901590,1338,Ashfield,MA,Greenfield Town,Franklin,2018-04-01,209300.0
3901591,3293,Woodstock,NH,Claremont,Grafton,2018-04-01,225800.0
3901592,40404,Berea,KY,Richmond,Madison,2018-04-01,133400.0
3901593,81225,Mount Crested Butte,CO,NaN,Gunnison,2018-04-01,664400.0


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results